In [3]:
!git clone https://github.com/monologg/KoBERT-NER.git

Cloning into 'KoBERT-NER'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 145 (delta 82), reused 97 (delta 38), pack-reused 0
Receiving objects: 100% (145/145), 17.54 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [4]:
# 저장소로 이동
%cd KoBERT-NER

/content/KoBERT-NER


In [ ]:
!pip install -r requirements.txt

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 학습 완료된 모델 불러오기

In [79]:
model_dir = '/content/drive/My Drive/monologg'  # 구글 드라이브에 monologg폴더를 만들고 학습완료된 모델을 업로드 (config.json, training_args.bin, pytorch_model.bin 업로드)

model = AutoModelForTokenClassification.from_pretrained(model_dir)
model.eval()
logger.info("***** Model Loaded *****")

11/01/2021 12:12:08 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/monologg/config.json
11/01/2021 12:12:08 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "naver-ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "UNK",
    "1": "O",
    "2": "PER-B",
    "3": "PER-I",
    "4": "FLD-B",
    "5": "FLD-I",
    "6": "AFW-B",
    "7": "AFW-I",
    "8": "ORG-B",
    "9": "ORG-I",
    "10": "LOC-B",
    "11": "LOC-I",
    "12": "CVL-B",
    "13": "CVL-I",
    "14": "DAT-B",
    "15": "DAT-I",
    "16": "TIM-B",
    "17": "TIM-I",
    "18": "NUM-B",
    "19": "NUM-I",
    "20": "EVT-B",
    "21": "EVT-I",
    "22": "ANM-B",
    "23": "ANM-I",
    "24": "PLT-B",
    "25": "PLT-I",
    "26": "MAT-B",
    "27": "MAT-I",
    "28": "TRM-B",
    "29": "

## KoBERT Tokenizer 불러오기

In [80]:
data_dir = '/content/KoBERT-NER/data'
label_file_dir = '/content/KoBERT-NER/data/label.txt'

label_lst =  [label.strip() for label in open(os.path.join(data_dir, label_file_dir), 'r', encoding='utf-8')]

In [81]:
from tokenization_kobert import KoBertTokenizer

args = torch.load(os.path.join('/content/drive/My Drive/monologg', 'training_args.bin'))
pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index
tokenizer = KoBertTokenizer.from_pretrained(args.model_name_or_path)

11/01/2021 12:12:12 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model from cache at /root/.cache/torch/transformers/1f871cf1d80a4c09742b5f2094cc7ab9163dfbcfb930d99fdc9149aebf6a13bb.7555cd8e0e341a4d19f33dcb20eaf8614c72f56827d1b28481046caabba18eb3
11/01/2021 12:12:12 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt from cache at /root/.cache/torch/transformers/c247d0225676d99ae7f3e6d39dba68b77756181415828f922c6393e4aecb53ce.5d8505da4be274704344000e8d9a798ff03b513f5053f98f7e97d719a9a9891b


## Dataloader

In [82]:
def read_input_file(ipath):
    lines = []
    with open(ipath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            words = line.split()
            lines.append(words)

    return lines

In [83]:
input_file = '/content/articles.txt'  # 입력 txt 파일을 업로드

lines = read_input_file(input_file)

In [84]:
 dataset = convert_input_file_to_tensor_dataset(lines, 'a', args, tokenizer, pad_token_label_id)

In [85]:
dataset

## Predict

In [86]:
sampler = SequentialSampler(dataset)
data_loader = DataLoader(dataset, sampler=sampler, batch_size=32)

all_slot_label_mask = None
preds = None

for batch in tqdm(data_loader, desc="Predicting"):
    batch = tuple(t for t in batch)
    with torch.no_grad():
        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": None}
        if args.model_type != "distilkobert":
            inputs["token_type_ids"] = batch[2]
        outputs = model(**inputs)
        logits = outputs[0]

        if preds is None:
            preds = logits.detach().cpu().numpy()
            all_slot_label_mask = batch[3].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            all_slot_label_mask = np.append(all_slot_label_mask, batch[3].detach().cpu().numpy(), axis=0)

Predicting: 100%|██████████| 8/8 [00:29<00:00,  3.66s/it]


In [95]:
len(lines)

249

In [87]:
preds.shape

(249, 50, 30)

## 결과 정리

In [88]:
preds = np.argmax(preds, axis=2)
slot_label_map = {i: label for i, label in enumerate(label_lst)}
preds_list = [[] for _ in range(preds.shape[0])]

for i in range(preds.shape[0]):
    for j in range(preds.shape[1]):
        if all_slot_label_mask[i, j] != pad_token_label_id:
            preds_list[i].append(slot_label_map[preds[i][j]])

In [98]:
res = {}
for words, preds in zip(lines, preds_list):
    for word, pred in zip(words, preds):
        if pred == 'O':
          if pred not in res.keys():
              res[pred] = [word]
          else:
              res[pred].append(word)
        else:
            if pred not in res.keys():
                res[pred] = [word]
            else:
                res[pred].append(word)

In [136]:
print(res)

{'CVL-B': ["'앵커", '기자의', "'기자", '한국우주환경과학연구소장', '미국인은', '미국인의', "'앵커", "'시청자들이", '기자가', "'기자", '시청자가', '교수', '교수', '공학도들이', '우주개발진흥법을', '선구자인', '과학자들은', '기자가', '군인', '신부와도', '공산주의자”라며', '우주인이', '우주인을', '국민과', '우주인의', '방문객만', '후발주자이지만', '과학기술자들의', '월동연구대원', "'대원들은", '옵서버로서', '연구원들이', '연구원들은', '책임연구원은', '원장은', '과학자들이', "'과학자들은", '교수', '연구단장', '교수', '연구단장', '교정연구단장이', '연구진은', '기자', '스마트디바이스본부장은', '소비자가', '기자', "'운전자가", '해킹', '미래학자이자', "군사다',", '저명인사들은', '공동창업자인', '최고경영자인', '박사는', '로봇성장사업단장은', '단장은', '직원수는', '운동화가', '소비자가', '근로자', '석·박사급', '연구자가', "'전문가들은", '회장은', '미국로봇산업협회장도', '리더가', '1차관은'], 'DAT-B': ['새해', "'4월에는", '8월에는', '올해', "'5년여마다", '4월', '8월의', "'38년", '8월', '오는', "'2017년의", "'새해", "'새해", '첫', "'5일", '4일간', '올해에는', '올해는', "첫해입니다',", '올해', '내년', '지난해', '10년간', '올해', '2013년', "'1995년", '올해', '2020년에', '2021년까지', '2018년까지', '2020년에는', "'올해", "'냉전시대와", '11월', "'1961년", "'1969년", "'21세기", "'하루", "'2015년", "'1992년부터", "'1992년", '2011년', '“20세기', "'2017", '새해를', "여름이다',", '1년간', '한여름에는'

In [137]:
print(res['CVL-B'])

["'앵커", '기자의', "'기자", '한국우주환경과학연구소장', '미국인은', '미국인의', "'앵커", "'시청자들이", '기자가', "'기자", '시청자가', '교수', '교수', '공학도들이', '우주개발진흥법을', '선구자인', '과학자들은', '기자가', '군인', '신부와도', '공산주의자”라며', '우주인이', '우주인을', '국민과', '우주인의', '방문객만', '후발주자이지만', '과학기술자들의', '월동연구대원', "'대원들은", '옵서버로서', '연구원들이', '연구원들은', '책임연구원은', '원장은', '과학자들이', "'과학자들은", '교수', '연구단장', '교수', '연구단장', '교정연구단장이', '연구진은', '기자', '스마트디바이스본부장은', '소비자가', '기자', "'운전자가", '해킹', '미래학자이자', "군사다',", '저명인사들은', '공동창업자인', '최고경영자인', '박사는', '로봇성장사업단장은', '단장은', '직원수는', '운동화가', '소비자가', '근로자', '석·박사급', '연구자가', "'전문가들은", '회장은', '미국로봇산업협회장도', '리더가', '1차관은']


In [160]:
tags = list(res.keys())
lines =''
for tag in tags:
    words = res[tag]
    line = f'SEP {tag} : '
    for word in words:
        line += ', ' + word

    lines += line

In [163]:
lines.split('SEP')

['',
 " CVL-B : , '앵커, 기자의, '기자, 한국우주환경과학연구소장, 미국인은, 미국인의, '앵커, '시청자들이, 기자가, '기자, 시청자가, 교수, 교수, 공학도들이, 우주개발진흥법을, 선구자인, 과학자들은, 기자가, 군인, 신부와도, 공산주의자”라며, 우주인이, 우주인을, 국민과, 우주인의, 방문객만, 후발주자이지만, 과학기술자들의, 월동연구대원, '대원들은, 옵서버로서, 연구원들이, 연구원들은, 책임연구원은, 원장은, 과학자들이, '과학자들은, 교수, 연구단장, 교수, 연구단장, 교정연구단장이, 연구진은, 기자, 스마트디바이스본부장은, 소비자가, 기자, '운전자가, 해킹, 미래학자이자, 군사다',, 저명인사들은, 공동창업자인, 최고경영자인, 박사는, 로봇성장사업단장은, 단장은, 직원수는, 운동화가, 소비자가, 근로자, 석·박사급, 연구자가, '전문가들은, 회장은, 미국로봇산업협회장도, 리더가, 1차관은",
 " DAT-B : , 새해, '4월에는, 8월에는, 올해, '5년여마다, 4월, 8월의, '38년, 8월, 오는, '2017년의, '새해, '새해, 첫, '5일, 4일간, 올해에는, 올해는, 첫해입니다',, 올해, 내년, 지난해, 10년간, 올해, 2013년, '1995년, 올해, 2020년에, 2021년까지, 2018년까지, 2020년에는, '올해, '냉전시대와, 11월, '1961년, '1969년, '21세기, '하루, '2015년, '1992년부터, '1992년, 2011년, “20세기, '2017, 새해를, 여름이다',, 1년간, 한여름에는, 5, 겨울에는, '세종기지는, 올해가, 내년이면, '올해, 병신년, 丙申年, 한, 지난달, 병신년, 丙申年, 한, 지난해, 지난해, 11월, '올해, 오는, 올해, 올, 올해, 지난해, 올해도, 지난해, 2014년, 50년, 오는, 오는, 2일부터, 지난해, '2017년, 올, '1일, 올, 지난해, 올해, “지난해, 내년에, '2011년부터, '25년, 2015년, '지난해, 2015년, 

In [169]:
# 예측 결과를 txt 파일로 저장
lines =''
with open('/content/preds_summary.txt', 'w') as file:  
    for tag in tags:
        words = res[tag]
        line = f'{tag} : '
        for word in words:
            line += ', ' + word

        file.write("{}\n".format(line.strip()))